In [ ]:
def tree_binning(df,target,var):
    
    
    """
    Parameters:
    -----------
        df : Dataframe for which Binning to be performed
        target : Target variable
        var : Numerical Variable for which binning to be performed
    Returns:
    --------
        Dataframe with the binned variable
    """
    
    
    tree_model = DecisionTreeClassifier(max_depth=3)
    df1=train_sample_raw[[target,var]].copy()
    df1.dropna(inplace=True)
    tree_model.fit(df1[var].to_frame(), df1[target])
    df1[var+"_tree"]=tree_model.predict_proba(df1[var].to_frame())[:,1]
    print('\n\n The Probability of events are : \n')
    print(df1[var+"_tree"].unique())
    print('\n \n The Bins created are : \n')
    df_bin=pd.concat( [df1.groupby([var+"_tree"])[var].min(),df1.groupby([var+"_tree"])[var].max()] ,axis=1)
    df_bin.columns=[var+"_min",var+"_max"]
    df_bin.sort_values(by=[var+'_min'],inplace=True)
    print(df_bin)
    df_bin[var+"_max"].iloc[-1]=np.inf
    print('\n\n The Bins after including right limit as infinity : \n')
    print(df_bin)
    bins = pd.IntervalIndex.from_tuples([(df_bin[var+'_min'][i],df_bin[var+'_max'][i]) for i in df_bin.index])
    print('\n\nThe bins created are : \n')
    print(bins)
    df[var+'_bin']=pd.cut(df[var],bins,include_lowest=True)
    print('\n\n The Count distribution of bins\n')
    print(df[var+'_bin'].value_counts())
    tree.plot_tree(tree_model)
    return (df)

In [ ]:
# Binning the train data variable D1

train_sample_raw=tree_binning(train_sample_raw,'isFraud','D1')


In [ ]:
# Binning the test data variable D1

test_sample_raw=tree_binning(test_sample_raw,'isFraud','D1')

In [ ]:
def  categorical_plots_with_target(df, variables, target, model='clf', add_missing = True):
    
    """
    Parameters:
    -----------
        df : Dataframe for which Analysis to be performed
        variables : input type list. All the categorical variables needed for plotting
        target : Target variable
        model : type of problem - classification or regression
                For classification related analysis. use 'classification' or 'clf'
                For regression related analysis. use 'regression' or 'reg'
        add_missing : default True. if True it will replace missing values by 'Missing'
        
    Returns:
    --------
        Category plots for all the variables
    
    """
    
    eda_df = df.copy(deep=True)
    length_df = len(eda_df)

    for i, var in enumerate(variables, 1):

        printmd (f'**<u> {i}. Plot for {var}</u>**')
        
        if add_missing:
            eda_df[var] = eda_df[var].cat.add_categories('Missing')
            eda_df[var] = eda_df[var].fillna('Missing')
        
        plot_df =  calculate_mean_target_per_category (eda_df, var, target)
        #print(plot_df)
        plot_df.sort_index(axis = 0,inplace=True) 
        #print(plot_df)
        cat_order = list(plot_df[var])
        #print(cat_order)

        if model in('clf' or 'classification'):
            plot_df[target] = 100*plot_df[target]

        

        fig, ax = plt.subplots(figsize=(12,4))
        plt.xticks(plot_df.index, plot_df[var], rotation = 90)

        ax.bar(plot_df.index, plot_df['perc'], align = 'center', color = 'lightgrey')
        
        
        ax2 = ax.twinx()
        ax2 = sns.pointplot(data = plot_df, x=var, y=target, order = cat_order, color='green')


        if model in('clf' or 'classification'):
            ax.set_title(f'Event Rate of target ({target}) within each category of variable ({var})', fontsize=17)
            ax2.set_ylabel("Perc of Events within Category", fontsize=14)
            #ax.set_xlabel(var, fontsize=14)
            ax.set_ylabel('Perc of Categories', fontsize=14)

        elif model in('reg' or 'regression'):
            ax.set_title(f'Mean value of target ({target}) within each category of variable ({var})', fontsize=17)
            ax2.set_ylabel('Mean Target Value', fontsize=14) 
            #ax.set_xlabel(var, fontsize=14)
            ax.set_ylabel('Perc of Categories', fontsize=14)
               

        plt.show()

        display_all(plot_df.set_index(var).transpose())

In [ ]:
# Plotting the variables after binning for train
categorical_plots_with_target(train_sample_raw, variables=['D1_bin'], target='isFraud', model='clf')

In [ ]:
# Plotting the variables after binning for test
categorical_plots_with_target(test_sample_raw, variables=['D1_bin'], target='isFraud', model='clf')